In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# 1. Preprocessing

In [ ]:
def load_data(file_path):
    """
    Carga los datos del archivo CSV con manejo de encoding
    
    Args:
        file_path (str): Ruta al archivo CSV
        
    Returns:
        pd.DataFrame: DataFrame con los datos cargados
    """
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        print(f"Datos cargados exitosamente con encoding UTF-8")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
        print(f"Datos cargados exitosamente con encoding ISO-8859-1")
    
    print(f"Forma del dataset: {df.shape}")
    return df

def remove_duplicates(df):
    """
    Elimina registros duplicados del dataset
    
    Args:
        df (pd.DataFrame): DataFrame original
        
    Returns:
        pd.DataFrame: DataFrame sin duplicados
    """
    initial_shape = df.shape[0]
    df_clean = df.drop_duplicates()
    final_shape = df_clean.shape[0]
    
    print(f"Registros duplicados eliminados: {initial_shape - final_shape}")
    print(f"Registros restantes: {final_shape}")
    
    return df_clean

def handle_null_customer_id(df):
    """
    Elimina registros con CustomerID nulo
    
    Args:
        df (pd.DataFrame): DataFrame original
        
    Returns:
        pd.DataFrame: DataFrame sin CustomerID nulos
    """
    initial_shape = df.shape[0]
    null_customers = df['CustomerID'].isnull().sum()
    
    print(f"Registros con CustomerID nulo: {null_customers}")
    
    if null_customers > 0:
        # Análisis por país antes de eliminar
        country_nulls = df.groupby('Country').agg({
            'CustomerID': lambda x: x.isnull().sum()
        }).query('CustomerID > 0').sort_values('CustomerID', ascending=False)
        
        print("Países con CustomerID nulos:")
        for country, nulls in country_nulls['CustomerID'].items():
            total = len(df[df['Country'] == country])
            percent = (nulls / total) * 100
            print(f"  {country}: {nulls} nulos de {total} transacciones ({percent:.1f}%)")
    
    df_clean = df[df['CustomerID'].notna()].copy()
    final_shape = df_clean.shape[0]
    
    print(f"Registros eliminados: {initial_shape - final_shape}")
    print(f"Registros restantes: {final_shape}")
    
    return df_clean

def filter_cancelled_transactions(df):
    """
    Filtra transacciones de cancelación (InvoiceNo que empiezan con 'C')
    
    Args:
        df (pd.DataFrame): DataFrame original
        
    Returns:
        pd.DataFrame: DataFrame sin transacciones canceladas
    """
    initial_shape = df.shape[0]
    cancelled_mask = df['InvoiceNo'].str.startswith('C', na=False)
    cancelled_count = cancelled_mask.sum()
    
    print(f"Transacciones de cancelación encontradas: {cancelled_count}")
    
    if cancelled_count > 0:
        # Análisis de las cancelaciones
        print("Análisis de transacciones canceladas:")
        cancelled_df = df[cancelled_mask]
        print(f"  Clientes únicos con cancelaciones: {cancelled_df['CustomerID'].nunique()}")
        print(f"  Países con cancelaciones: {cancelled_df['Country'].nunique()}")
        print(f"  Total quantity en cancelaciones: {cancelled_df['Quantity'].sum()}")
    
    df_clean = df[~cancelled_mask].copy()
    final_shape = df_clean.shape[0]
    
    print(f"Registros eliminados: {initial_shape - final_shape}")
    print(f"Registros restantes: {final_shape}")
    
    return df_clean

def handle_inconsistent_data(df):
    """
    Maneja datos inconsistentes: Quantity negativa y UnitPrice = 0
    
    Args:
        df (pd.DataFrame): DataFrame original
        
    Returns:
        pd.DataFrame: DataFrame con datos consistentes
    """
    initial_shape = df.shape[0]
    
    # Analizar quantity negativa
    negative_quantity = (df['Quantity'] < 0).sum()
    zero_quantity = (df['Quantity'] == 0).sum()
    
    # Analizar UnitPrice = 0 o negativo
    zero_price = (df['UnitPrice'] == 0).sum()
    negative_price = (df['UnitPrice'] < 0).sum()
    
    print(f"Análisis de datos inconsistentes:")
    print(f"  Quantity negativa: {negative_quantity}")
    print(f"  Quantity igual a 0: {zero_quantity}")
    print(f"  UnitPrice igual a 0: {zero_price}")
    print(f"  UnitPrice negativo: {negative_price}")
    
    # Filtrar datos inconsistentes
    consistent_mask = (
        (df['Quantity'] > 0) &
        (df['UnitPrice'] > 0)
    )
    
    df_clean = df[consistent_mask].copy()
    final_shape = df_clean.shape[0]
    
    print(f"Registros eliminados: {initial_shape - final_shape}")
    print(f"Registros restantes: {final_shape}")
    
    return df_clean

def balance_geographical_data(df):
    """
    Balancea los datos geográficos eliminando países con pocas transacciones
    
    Args:
        df (pd.DataFrame): DataFrame original
        
    Returns:
        pd.DataFrame: DataFrame balanceado geográficamente
    """
    # Clasificar países por volumen de transacciones
    country_stats = df['Country'].value_counts()
    
    # Definir tiers
    tier1 = country_stats[country_stats >= 1000].index.tolist()
    tier2 = country_stats[(country_stats >= 100) & (country_stats < 1000)].index.tolist()
    tier3 = country_stats[country_stats < 100].index.tolist()
    
    print(f"Clasificación por tiers:")
    print(f"  Tier 1 (>=1000 trans): {len(tier1)} países - {country_stats[tier1].sum():,} transacciones")
    print(f"  Tier 2 (100-999 trans): {len(tier2)} países - {country_stats[tier2].sum():,} transacciones")
    print(f"  Tier 3 (<100 trans): {len(tier3)} países - {country_stats[tier3].sum():,} transacciones")
    
    # Asignar tiers
    def assign_tier(country):
        if country in tier1:
            return 'Tier1_Principal'
        elif country in tier2:
            return 'Tier2_Mediano'
        else:
            return 'Tier3_Pequeno'
    
    df['Country_Tier'] = df['Country'].apply(assign_tier)
    
    # Asignar regiones
    def assign_region(country):
        europe = ['United Kingdom', 'Germany', 'France', 'Spain', 'Netherlands', 
                  'Belgium', 'Switzerland', 'Austria', 'Italy', 'Portugal', 'Norway',
                  'Denmark', 'Finland', 'Sweden', 'Poland', 'Cyprus']
        
        asia_pacific = ['Australia', 'Japan', 'Singapore', 'Hong Kong']
        americas = ['USA', 'Canada', 'Brazil']
        
        if country in europe:
            return 'Europa'
        elif country in asia_pacific:
            return 'Asia_Pacifico'
        elif country in americas:
            return 'Americas'
        else:
            return 'Otros'
    
    df['Region'] = df['Country'].apply(assign_region)
    
    # Filtrar para mantener balance
    initial_shape = df.shape[0]
    df_balanced = df[
        (df['Country_Tier'].isin(['Tier1_Principal', 'Tier2_Mediano'])) &
        (df['Region'].isin(['Europa', 'Asia_Pacifico', 'Americas']))
    ].copy()
    
    final_shape = df_balanced.shape[0]
    
    print(f"Filtrado geográfico aplicado:")
    print(f"  Registros eliminados: {initial_shape - final_shape}")
    print(f"  Registros restantes: {final_shape}")
    print(f"  Porcentaje mantenido: {(final_shape / initial_shape) * 100:.1f}%")
    
    return df_balanced

def handle_skewed_data(df):
    """
    Maneja datos sesgados aplicando transformaciones
    
    Args:
        df (pd.DataFrame): DataFrame original
        
    Returns:
        pd.DataFrame: DataFrame con transformaciones aplicadas
    """
    print("Análisis de sesgo en UnitPrice:")
    
    # Estadísticas originales
    original_skew = stats.skew(df['UnitPrice'])
    original_kurtosis = stats.kurtosis(df['UnitPrice'])
    
    print(f"  Sesgo original: {original_skew:.3f}")
    print(f"  Curtosis original: {original_kurtosis:.3f}")
    
    # Transformación logarítmica
    df['UnitPrice_Log'] = np.log1p(df['UnitPrice'])
    log_skew = stats.skew(df['UnitPrice_Log'])
    
    # Winsorización (outliers extremos)
    p99 = df['UnitPrice'].quantile(0.99)
    p1 = df['UnitPrice'].quantile(0.01)
    df['UnitPrice_Winsorized'] = df['UnitPrice'].clip(lower=p1, upper=p99)
    winsor_skew = stats.skew(df['UnitPrice_Winsorized'])
    
    # Crear variable de valor total
    df['TotalValue'] = df['Quantity'] * df['UnitPrice']
    df['TotalValue_Log'] = np.log1p(df['TotalValue'].clip(lower=0))
    
    # Segmentos de precios
    df['Price_Segment'] = pd.cut(df['UnitPrice'], 
                                bins=[0, 1, 5, 20, np.inf], 
                                labels=['Bajo', 'Medio', 'Alto', 'Premium'])
    
    print(f"Transformaciones aplicadas:")
    print(f"  Sesgo log-transformado: {log_skew:.3f}")
    print(f"  Sesgo winsorizado: {winsor_skew:.3f}")
    print(f"  Mejora en sesgo (log): {((original_skew - log_skew) / original_skew * 100):.1f}%")
    
    return df

def validate_data_quality(df):
    """
    Valida la calidad del dataset procesado
    
    Args:
        df (pd.DataFrame): DataFrame procesado
        
    Returns:
        dict: Métricas de calidad
    """
    print("=== VALIDACIÓN DE CALIDAD DE DATOS ===")
    
    # Completitud
    print(f"Filas: {df.shape[0]:,}")
    print(f"Columnas: {df.shape[1]}")
    print(f"Sin duplicados: {df.duplicated().sum() == 0}")
    print(f"Sin CustomerID nulos: {df['CustomerID'].isnull().sum() == 0}")
    
    # Consistencia
    print(f"Quantity > 0: {(df['Quantity'] > 0).all()}")
    print(f"UnitPrice > 0: {(df['UnitPrice'] > 0).all()}")
    print(f"Sin transacciones canceladas: {not df['InvoiceNo'].str.startswith('C', na=False).any()}")
    
    # Integridad temporal
    if 'InvoiceDate' in df.columns:
        df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
        date_range = (df['InvoiceDate'].max() - df['InvoiceDate'].min()).days
        print(f"Rango temporal: {df['InvoiceDate'].min()} a {df['InvoiceDate'].max()}")
        print(f"Días de datos: {date_range}")
    
    # Integridad de clientes
    customer_stats = df.groupby('CustomerID').agg({
        'InvoiceNo': 'nunique',
        'TotalValue': 'sum'
    }).describe()
    
    print(f"Clientes únicos: {df['CustomerID'].nunique():,}")
    print(f"Transacciones por cliente (promedio): {customer_stats['InvoiceNo']['mean']:.1f}")
    print(f"Valor promedio por cliente: ${customer_stats['TotalValue']['mean']:.2f}")
    
    # Distribución geográfica
    if 'Country_Tier' in df.columns and 'Region' in df.columns:
        print(f"\nDistribución por tiers:")
        tier_dist = df['Country_Tier'].value_counts()
        for tier, count in tier_dist.items():
            print(f"  {tier}: {count:,} ({count/len(df)*100:.1f}%)")
        
        print(f"\nDistribución por regiones:")
        region_dist = df['Region'].value_counts()
        for region, count in region_dist.items():
            print(f"  {region}: {count:,} ({count/len(df)*100:.1f}%)")
    
    return {
        'total_rows': df.shape[0],
        'total_columns': df.shape[1],
        'unique_customers': df['CustomerID'].nunique(),
        'date_range_days': date_range if 'InvoiceDate' in df.columns else None,
        'data_quality_score': 100  # Simplificado, se puede expandir
    }

def preprocess_retail_data(file_path, save_path=None):
    """
    Pipeline completo de preprocesamiento de datos retail
    
    Args:
        file_path (str): Ruta al archivo de datos original
        save_path (str, optional): Ruta para guardar datos procesados
        
    Returns:
        pd.DataFrame: Dataset procesado
    """
    print("=== INICIANDO PIPELINE DE PREPROCESAMIENTO ===\n")
    
    # 1. Cargar datos
    print("PASO 1: Cargando datos...")
    df = load_data(file_path)
    print()
    
    # 2. Remover duplicados
    print("PASO 2: Removiendo duplicados...")
    df = remove_duplicates(df)
    print()
    
    # 3. Manejar CustomerID nulos
    print("PASO 3: Manejando CustomerID nulos...")
    df = handle_null_customer_id(df)
    print()
    
    # 4. Filtrar cancelaciones
    print("PASO 4: Filtrando transacciones canceladas...")
    df = filter_cancelled_transactions(df)
    print()
    
    # 5. Manejar datos inconsistentes
    print("PASO 5: Manejando datos inconsistentes...")
    df = handle_inconsistent_data(df)
    print()
    
    # 6. Balancear datos geográficos
    print("PASO 6: Balanceando datos geográficos...")
    df = balance_geographical_data(df)
    print()
    
    # 7. Manejar datos sesgados
    print("PASO 7: Manejando datos sesgados...")
    df = handle_skewed_data(df)
    print()
    
    # 8. Validar calidad
    print("PASO 8: Validando calidad de datos...")
    quality_metrics = validate_data_quality(df)
    print()
    
    # 9. Guardar si se especifica ruta
    if save_path:
        df.to_csv(save_path, index=False)
        print(f"Datos procesados guardados en: {save_path}")
    
    print("=== PIPELINE COMPLETADO EXITOSAMENTE ===")
    return df

In [ ]:
df_processed = preprocess_retail_data(
    file_path='../data/transacciones_retail.csv',
)

In [ ]:
df_processed

# 2. Feature Engineering

In [ ]:
def create_rfm_features(df, analysis_date=None):
    """
    Crea características RFM (Recency, Frequency, Monetary) para cada cliente
    
    Args:
        df (pd.DataFrame): DataFrame con datos transaccionales
        analysis_date (str, optional): Fecha de análisis. Si no se proporciona, usa la fecha máxima
        
    Returns:
        pd.DataFrame: DataFrame con métricas RFM por cliente
    """
    print("=== CREANDO CARACTERÍSTICAS RFM ===")
    
    # Asegurar que InvoiceDate sea datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    
    # Definir fecha de análisis
    if analysis_date is None:
        analysis_date = df['InvoiceDate'].max()
    else:
        analysis_date = pd.to_datetime(analysis_date)
    
    print(f"Fecha de análisis: {analysis_date}")
    
    # Calcular métricas RFM por cliente
    rfm_data = df.groupby('CustomerID').agg({
        'InvoiceDate': lambda x: (analysis_date - x.max()).days,  # Recency
        'InvoiceNo': 'nunique',  # Frequency
        'TotalValue': 'sum'  # Monetary
    }).reset_index()
    
    # Renombrar columnas
    rfm_data.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
    
    # Crear características adicionales
    rfm_data['AvgOrderValue'] = rfm_data['Monetary'] / rfm_data['Frequency']
    rfm_data['DaysActive'] = df.groupby('CustomerID')['InvoiceDate'].apply(
        lambda x: (x.max() - x.min()).days
    ).values
    
    # Métricas adicionales de comportamiento
    customer_behavior = df.groupby('CustomerID').agg({
        'Quantity': ['sum', 'mean', 'std'],
        'UnitPrice': ['mean', 'std'],
        'Country': lambda x: x.mode()[0],  # País más frecuente
        'StockCode': 'nunique'  # Variedad de productos
    }).reset_index()
    
    # Aplanar nombres de columnas
    customer_behavior.columns = [
        'CustomerID', 'TotalQuantity', 'AvgQuantity', 'StdQuantity',
        'AvgUnitPrice', 'StdUnitPrice', 'MostFrequentCountry', 'ProductVariety'
    ]
    
    # Combinar con RFM
    rfm_features = rfm_data.merge(customer_behavior, on='CustomerID', how='left')
    
    # Manejar valores nulos en desviaciones estándar
    rfm_features['StdQuantity'] = rfm_features['StdQuantity'].fillna(0)
    rfm_features['StdUnitPrice'] = rfm_features['StdUnitPrice'].fillna(0)
    
    print(f"Características RFM creadas para {len(rfm_features)} clientes")
    
    return rfm_features

def create_rfm_scores(rfm_features):
    """
    Crea scores RFM usando quintiles
    
    Args:
        rfm_features (pd.DataFrame): DataFrame con características RFM
        
    Returns:
        pd.DataFrame: DataFrame con scores RFM añadidos
    """
    print("=== CREANDO SCORES RFM ===")
    
    rfm_scored = rfm_features.copy()
    
    # Crear scores usando quintiles (1-5)
    # Recency: Score más alto para menor recencia (más reciente = mejor)
    rfm_scored['R_Score'] = pd.qcut(rfm_scored['Recency'], 5, labels=[5,4,3,2,1])
    
    # Frequency: Score más alto para mayor frecuencia
    rfm_scored['F_Score'] = pd.qcut(rfm_scored['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
    
    # Monetary: Score más alto para mayor valor monetario
    rfm_scored['M_Score'] = pd.qcut(rfm_scored['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])
    
    # Crear score RFM combinado
    rfm_scored['RFM_Score'] = (
        rfm_scored['R_Score'].astype(str) + 
        rfm_scored['F_Score'].astype(str) + 
        rfm_scored['M_Score'].astype(str)
    )
    
    # Crear segmentos de clientes
    def assign_customer_segment(row):
        r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])
        
        if r >= 4 and f >= 4 and m >= 4:
            return 'Champions'
        elif r >= 3 and f >= 3 and m >= 3:
            return 'Loyal_Customers'
        elif r >= 4 and f <= 2:
            return 'New_Customers'
        elif r >= 3 and f <= 2 and m >= 3:
            return 'Potential_Loyalists'
        elif r <= 2 and f >= 3 and m >= 3:
            return 'At_Risk'
        elif r <= 2 and f <= 2 and m >= 3:
            return 'Cannot_Lose'
        elif r >= 3 and f >= 3 and m <= 2:
            return 'Price_Sensitive'
        else:
            return 'Others'
    
    rfm_scored['Customer_Segment'] = rfm_scored.apply(assign_customer_segment, axis=1)
    
    # Estadísticas de segmentos
    segment_stats = rfm_scored['Customer_Segment'].value_counts()
    print("Distribución de segmentos de clientes:")
    for segment, count in segment_stats.items():
        print(f"  {segment}: {count:,} ({count/len(rfm_scored)*100:.1f}%)")
    
    return rfm_scored

def create_target_variable(df, prediction_days=90, analysis_date=None):
    """
    Crea variable objetivo: si el cliente volverá a comprar en los próximos X días
    
    Args:
        df (pd.DataFrame): DataFrame con datos transaccionales
        prediction_days (int): Días hacia el futuro para la predicción
        analysis_date (str, optional): Fecha de corte para el análisis
        
    Returns:
        pd.DataFrame: DataFrame con variable objetivo por cliente
    """
    print(f"=== CREANDO VARIABLE OBJETIVO ({prediction_days} días) ===")
    
    # Asegurar que InvoiceDate sea datetime
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    
    # Definir fecha de corte
    if analysis_date is None:
        # Usar una fecha que permita tener datos futuros para validar
        analysis_date = df['InvoiceDate'].quantile(0.7)  # 70% de los datos para entrenamiento
    else:
        analysis_date = pd.to_datetime(analysis_date)
    
    print(f"Fecha de corte para análisis: {analysis_date}")
    print(f"Fecha límite para predicción: {analysis_date + pd.Timedelta(days=prediction_days)}")
    
    # Separar datos históricos y futuros
    historical_data = df[df['InvoiceDate'] <= analysis_date]
    future_data = df[df['InvoiceDate'] > analysis_date]
    
    print(f"Registros históricos: {len(historical_data):,}")
    print(f"Registros futuros: {len(future_data):,}")
    
    # Clientes que aparecen en datos históricos
    historical_customers = set(historical_data['CustomerID'].unique())
    
    # Clientes que compran en el período de predicción
    future_cutoff = analysis_date + pd.Timedelta(days=prediction_days)
    prediction_period_data = future_data[
        (future_data['InvoiceDate'] > analysis_date) & 
        (future_data['InvoiceDate'] <= future_cutoff)
    ]
    
    future_customers = set(prediction_period_data['CustomerID'].unique())
    
    print(f"Clientes en período histórico: {len(historical_customers):,}")
    print(f"Clientes que compran en período de predicción: {len(future_customers):,}")
    
    # Crear variable objetivo
    target_data = []
    
    for customer_id in historical_customers:
        will_purchase = 1 if customer_id in future_customers else 0
        target_data.append({
            'CustomerID': customer_id,
            'WillPurchase_90Days': will_purchase
        })
    
    target_df = pd.DataFrame(target_data)
    
    # Estadísticas de la variable objetivo
    target_distribution = target_df['WillPurchase_90Days'].value_counts()
    print(f"\nDistribución de variable objetivo:")
    print(f"  No volverá a comprar (0): {target_distribution[0]:,} ({target_distribution[0]/len(target_df)*100:.1f}%)")
    print(f"  Volverá a comprar (1): {target_distribution[1]:,} ({target_distribution[1]/len(target_df)*100:.1f}%)")
    
    return target_df, analysis_date

def create_temporal_features(df, analysis_date):
    """
    Crea características temporales adicionales
    
    Args:
        df (pd.DataFrame): DataFrame con datos transaccionales
        analysis_date (datetime): Fecha de corte para el análisis
        
    Returns:
        pd.DataFrame: DataFrame con características temporales por cliente
    """
    print("=== CREANDO CARACTERÍSTICAS TEMPORALES ===")
    
    # Filtrar solo datos históricos
    historical_data = df[df['InvoiceDate'] <= analysis_date].copy()
    
    # Crear características temporales
    historical_data['Year'] = historical_data['InvoiceDate'].dt.year
    historical_data['Month'] = historical_data['InvoiceDate'].dt.month
    historical_data['DayOfWeek'] = historical_data['InvoiceDate'].dt.dayofweek
    historical_data['Quarter'] = historical_data['InvoiceDate'].dt.quarter
    
    # Características de comportamiento temporal por cliente
    temporal_features = historical_data.groupby('CustomerID').agg({
        'InvoiceDate': [
            lambda x: x.nunique(),  # Días únicos de compra
            lambda x: (analysis_date - x.min()).days,  # Antigüedad del cliente
            lambda x: (x.max() - x.min()).days if len(x) > 1 else 0  # Período de actividad
        ],
        'Year': lambda x: x.nunique(),  # Años únicos de compra
        'Month': lambda x: x.nunique(),  # Meses únicos de compra
        'DayOfWeek': lambda x: x.mode()[0] if len(x) > 0 else 0,  # Día preferido
        'Quarter': lambda x: x.nunique()  # Trimestres únicos
    }).reset_index()
    
    # Aplanar nombres de columnas
    temporal_features.columns = [
        'CustomerID', 'UniquePurchaseDays', 'CustomerAge_Days', 'ActivityPeriod_Days',
        'UniqueYears', 'UniqueMonths', 'PreferredDayOfWeek', 'UniqueQuarters'
    ]
    
    # Calcular frecuencia de compra promedio
    temporal_features['AvgDaysBetweenPurchases'] = (
        temporal_features['ActivityPeriod_Days'] / 
        (temporal_features['UniquePurchaseDays'] - 1)
    ).fillna(0)
    
    print(f"Características temporales creadas para {len(temporal_features)} clientes")
    
    return temporal_features

def combine_all_features(rfm_features, target_df, temporal_features):
    """
    Combina todas las características en un dataset final
    
    Args:
        rfm_features (pd.DataFrame): Características RFM con scores
        target_df (pd.DataFrame): Variable objetivo
        temporal_features (pd.DataFrame): Características temporales
        
    Returns:
        pd.DataFrame: Dataset final con todas las características
    """
    print("=== COMBINANDO TODAS LAS CARACTERÍSTICAS ===")
    
    # Combinar todas las características
    final_dataset = rfm_features.merge(target_df, on='CustomerID', how='inner')
    final_dataset = final_dataset.merge(temporal_features, on='CustomerID', how='left')
    
    # Manejar valores nulos en características temporales
    temporal_cols = ['UniquePurchaseDays', 'CustomerAge_Days', 'ActivityPeriod_Days',
                    'UniqueYears', 'UniqueMonths', 'PreferredDayOfWeek', 'UniqueQuarters',
                    'AvgDaysBetweenPurchases']
    
    for col in temporal_cols:
        if col in final_dataset.columns:
            final_dataset[col] = final_dataset[col].fillna(0)
    
    # Convertir scores RFM a numérico
    final_dataset['R_Score'] = final_dataset['R_Score'].astype(int)
    final_dataset['F_Score'] = final_dataset['F_Score'].astype(int)
    final_dataset['M_Score'] = final_dataset['M_Score'].astype(int)
    
    print(f"Dataset final creado:")
    print(f"  Filas: {len(final_dataset):,}")
    print(f"  Columnas: {final_dataset.shape[1]}")
    print(f"  Clientes únicos: {final_dataset['CustomerID'].nunique():,}")
    
    return final_dataset

def feature_engineering_pipeline(df, prediction_days=90, save_path=None):
    """
    Pipeline completo de feature engineering
    
    Args:
        df (pd.DataFrame): DataFrame procesado con datos transaccionales
        prediction_days (int): Días para predicción futura
        save_path (str, optional): Ruta para guardar el dataset final
        
    Returns:
        pd.DataFrame: Dataset final con todas las características
    """
    print("=== INICIANDO PIPELINE DE FEATURE ENGINEERING ===\n")
    
    # 1. Crear características RFM
    print("PASO 1: Creando características RFM...")
    rfm_features = create_rfm_features(df)
    print()
    
    # 2. Crear scores RFM
    print("PASO 2: Creando scores RFM...")
    rfm_features = create_rfm_scores(rfm_features)
    print()
    
    # 3. Crear variable objetivo
    print("PASO 3: Creando variable objetivo...")
    target_df, analysis_date = create_target_variable(df, prediction_days)
    print()
    
    # 4. Crear características temporales
    print("PASO 4: Creando características temporales...")
    temporal_features = create_temporal_features(df, analysis_date)
    print()
    
    # 5. Combinar todas las características
    print("PASO 5: Combinando características...")
    final_dataset = combine_all_features(rfm_features, target_df, temporal_features)
    print()
    
    # 6. Guardar si se especifica ruta
    if save_path:
        final_dataset.to_csv(save_path, index=False)
        print(f"Dataset con feature engineering guardado en: {save_path}")
    
    print("=== PIPELINE DE FEATURE ENGINEERING COMPLETADO ===")
    return final_dataset

In [ ]:
df_features = feature_engineering_pipeline(
    df_processed,
    prediction_days=90,
    save_path='../data/retail_features_dataset.csv'
)

In [ ]:
print(f"\nDataset final con feature engineering:")
print(f"Forma: {df_features.shape}")
print(f"\nColumnas disponibles:")
for i, col in enumerate(df_features.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
print(f"\nPrimeras 5 filas:")
df_features.head()